In [1]:
import os
import numpy as np
from os.path import isfile, join
from pathlib import Path
import pandas as pd
#from ipynb.fs.full.BB_functions import files_from_folder, preprocess

In [2]:
input_path = Path(r'C:\Users\UCPH\gabi\thesis analysis\BB_results')
save_path = Path(r'C:\Users\UCPH\gabi\thesis analysis')

In [5]:
df = files_from_folder(input_path)

In [6]:
df

1,Nose_x,FrontLimb_x,HindLimb_x,TailBase_x,Start_x,End_x,Nose_y,FrontLimb_y,HindLimb_y,TailBase_y,Start_y,End_y,Nose_likelihood,FrontLimb_likelihood,HindLimb_likelihood,TailBase_likelihood,Start_likelihood,End_likelihood,Name File
3,143.545715,96.245529,44.022476,18.651905,20.583385,1117.812500,227.071884,233.196106,234.609909,205.949783,233.515091,217.730255,0.999998,0.999755,0.999822,0.998261,0.906770,0.999751,"1_POST_BB_15_H0,1_KI_M_T1"
4,175.072388,131.115768,47.533272,48.185768,24.782099,1117.903687,227.889694,232.324905,231.925858,210.241562,234.788910,217.909409,0.999998,0.999913,0.958160,0.999963,0.999372,0.999770,"1_POST_BB_15_H0,1_KI_M_T1"
5,175.080826,131.117554,47.529694,48.184433,24.792465,1117.905640,227.821259,232.311676,231.933731,210.271851,234.817993,217.910217,0.999998,0.999914,0.959692,0.999964,0.999352,0.999770,"1_POST_BB_15_H0,1_KI_M_T1"
6,197.428925,135.007690,84.884621,75.856750,28.166096,1118.107178,229.624435,233.597336,231.016907,205.154587,228.720490,217.636536,1.000000,0.999600,0.996195,0.999563,0.988480,0.999771,"1_POST_BB_15_H0,1_KI_M_T1"
7,213.881226,164.866318,106.001846,83.434853,27.849884,1118.424194,224.670609,230.557800,230.433044,208.147385,227.036591,218.671371,0.999999,0.999967,0.999543,0.999986,0.993718,0.999826,"1_POST_BB_15_H0,1_KI_M_T1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1173.244995,1137.593506,1079.978638,1054.612427,25.086863,1118.189453,243.692307,238.108337,216.358856,186.869125,238.498611,221.671524,1.000000,0.999294,0.999065,0.999747,0.999055,0.999399,"1_POST_BB_15_H0,1_KI_M_T3"
60,1173.224365,1137.629395,1079.887817,1054.536011,25.158651,1118.060181,243.640884,238.056854,216.364380,186.775192,238.246887,221.669830,1.000000,0.999319,0.999282,0.999760,0.999267,0.999399,"1_POST_BB_15_H0,1_KI_M_T3"
61,1178.074097,1139.895142,1086.941040,1073.521484,24.988279,1119.106567,247.164917,242.008514,217.300476,194.386749,238.995026,221.833191,0.999992,0.999803,0.999388,0.999971,0.999101,0.998947,"1_POST_BB_15_H0,1_KI_M_T3"
62,1185.117798,1142.025879,1107.346436,1086.742920,25.150826,1117.873047,242.263367,242.423752,226.647827,208.698837,238.373352,220.211594,0.999992,0.999941,0.508760,0.999950,0.999295,0.994867,"1_POST_BB_15_H0,1_KI_M_T3"


In [7]:
# Split the 'Name File' column into multiple columns
df[['Cohort', 'Pre-Post', 'Test_Type', 'Animal_ID', 'Drug_Dose', 'Genotype', 'Gender', 'Trial']] = df['Name File'].str.split('_', expand=True)

# Drop the original 'Name File' column if needed
# df.drop('Name File', axis=1, inplace=True)

# Split 'drug_dose' into 'Drug' and 'Dose' columns
df[['Drug', 'Dose']] = df['Drug_Dose'].str.extract(r'([A-Za-z]+)(\d+,\d+)')

# Extract only numbers from the 'Trial' column
df['Trial'] = df['Trial'].str.extract('(\d+)')

# Print the updated DataFrame
df

1,Nose_x,FrontLimb_x,HindLimb_x,TailBase_x,Start_x,End_x,Nose_y,FrontLimb_y,HindLimb_y,TailBase_y,...,Cohort,Pre-Post,Test_Type,Animal_ID,Drug_Dose,Genotype,Gender,Trial,Drug,Dose
3,143.545715,96.245529,44.022476,18.651905,20.583385,1117.812500,227.071884,233.196106,234.609909,205.949783,...,1,POST,BB,15,"H0,1",KI,M,1,H,"0,1"
4,175.072388,131.115768,47.533272,48.185768,24.782099,1117.903687,227.889694,232.324905,231.925858,210.241562,...,1,POST,BB,15,"H0,1",KI,M,1,H,"0,1"
5,175.080826,131.117554,47.529694,48.184433,24.792465,1117.905640,227.821259,232.311676,231.933731,210.271851,...,1,POST,BB,15,"H0,1",KI,M,1,H,"0,1"
6,197.428925,135.007690,84.884621,75.856750,28.166096,1118.107178,229.624435,233.597336,231.016907,205.154587,...,1,POST,BB,15,"H0,1",KI,M,1,H,"0,1"
7,213.881226,164.866318,106.001846,83.434853,27.849884,1118.424194,224.670609,230.557800,230.433044,208.147385,...,1,POST,BB,15,"H0,1",KI,M,1,H,"0,1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1173.244995,1137.593506,1079.978638,1054.612427,25.086863,1118.189453,243.692307,238.108337,216.358856,186.869125,...,1,POST,BB,15,"H0,1",KI,M,3,H,"0,1"
60,1173.224365,1137.629395,1079.887817,1054.536011,25.158651,1118.060181,243.640884,238.056854,216.364380,186.775192,...,1,POST,BB,15,"H0,1",KI,M,3,H,"0,1"
61,1178.074097,1139.895142,1086.941040,1073.521484,24.988279,1119.106567,247.164917,242.008514,217.300476,194.386749,...,1,POST,BB,15,"H0,1",KI,M,3,H,"0,1"
62,1185.117798,1142.025879,1107.346436,1086.742920,25.150826,1117.873047,242.263367,242.423752,226.647827,208.698837,...,1,POST,BB,15,"H0,1",KI,M,3,H,"0,1"


In [8]:
# Iterate through columns and set values to NaN based on condition
for col in df.columns:
    if col.endswith('_likelihood'):
        mask = df[col] < 0.7
        prefix = col[:-11]  # Remove '_likelihood' to get the prefix
        for suffix in ['x', 'y']:
            column_to_set_nan = f'{prefix}_{suffix}'
            df.loc[mask, column_to_set_nan] = np.nan

In [9]:
# Function to calculate the adjusted mean
def calculate_adjusted_mean(group):
    filtered_start_x = group[group['Start_likelihood'] > 0.95]['Start_x']
    filtered_start_y = group[group['Start_likelihood'] > 0.95]['Start_y']
    filtered_end_x = group[group['End_likelihood'] > 0.95]['End_x']
    filtered_end_y = group[group['End_likelihood'] > 0.95]['End_y']
    
    start_x_adjusted_mean = filtered_start_x.mean()
    start_y_adjusted_mean = filtered_start_y.mean()
    end_x_adjusted_mean = filtered_end_x.mean()
    end_y_adjusted_mean = filtered_end_y.mean()
    
    group['Start_x_adjust'] = start_x_adjusted_mean
    group['Start_y_adjust'] = start_y_adjusted_mean
    group['End_x_adjust'] = end_x_adjusted_mean
    group['End_y_adjust'] = end_y_adjusted_mean
    
    return group

# Group by 'Name File' and apply the calculation
df = df.groupby('Name File').apply(calculate_adjusted_mean)

# Fill NaN values in the new columns with appropriate values
df['Start_x_adjust'].fillna(df['Start_x'], inplace=True)
df['Start_y_adjust'].fillna(df['Start_y'], inplace=True)
df['End_x_adjust'].fillna(df['End_x'], inplace=True)
df['End_y_adjust'].fillna(df['End_y'], inplace=True)

# Drop the original 'Start_likelihood' and 'End_likelihood' columns if needed
# df.drop(['Start_likelihood', 'End_likelihood'], axis=1, inplace=True)

# Print the updated DataFrame
df

C:\Users\UCPH\AppData\Local\Temp\ipykernel_23840\3720783363.py:21: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Name File').apply(calculate_adjusted_mean)


1,Nose_x,FrontLimb_x,HindLimb_x,TailBase_x,Start_x,End_x,Nose_y,FrontLimb_y,HindLimb_y,TailBase_y,...,Drug_Dose,Genotype,Gender,Trial,Drug,Dose,Start_x_adjust,Start_y_adjust,End_x_adjust,End_y_adjust
3,143.545715,96.245529,44.022476,18.651905,20.583385,1117.812500,227.071884,233.196106,234.609909,205.949783,...,"H0,1",KI,M,1,H,"0,1",25.302628,235.666721,1118.057558,219.110961
4,175.072388,131.115768,47.533272,48.185768,24.782099,1117.903687,227.889694,232.324905,231.925858,210.241562,...,"H0,1",KI,M,1,H,"0,1",25.302628,235.666721,1118.057558,219.110961
5,175.080826,131.117554,47.529694,48.184433,24.792465,1117.905640,227.821259,232.311676,231.933731,210.271851,...,"H0,1",KI,M,1,H,"0,1",25.302628,235.666721,1118.057558,219.110961
6,197.428925,135.007690,84.884621,75.856750,28.166096,1118.107178,229.624435,233.597336,231.016907,205.154587,...,"H0,1",KI,M,1,H,"0,1",25.302628,235.666721,1118.057558,219.110961
7,213.881226,164.866318,106.001846,83.434853,27.849884,1118.424194,224.670609,230.557800,230.433044,208.147385,...,"H0,1",KI,M,1,H,"0,1",25.302628,235.666721,1118.057558,219.110961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1173.244995,1137.593506,1079.978638,1054.612427,25.086863,1118.189453,243.692307,238.108337,216.358856,186.869125,...,"H0,1",KI,M,3,H,"0,1",24.913291,236.915525,1118.056741,220.290889
60,1173.224365,1137.629395,1079.887817,1054.536011,25.158651,1118.060181,243.640884,238.056854,216.364380,186.775192,...,"H0,1",KI,M,3,H,"0,1",24.913291,236.915525,1118.056741,220.290889
61,1178.074097,1139.895142,1086.941040,1073.521484,24.988279,1119.106567,247.164917,242.008514,217.300476,194.386749,...,"H0,1",KI,M,3,H,"0,1",24.913291,236.915525,1118.056741,220.290889
62,1185.117798,1142.025879,NaN,1086.742920,25.150826,1117.873047,242.263367,242.423752,NaN,208.698837,...,"H0,1",KI,M,3,H,"0,1",24.913291,236.915525,1118.056741,220.290889


In [10]:
# Define a function to determine Framerate and y_pixels based on Cohort
def set_framerate_and_ypixels(row):
    if row['Cohort'] == 1:
        return 15, 720
    elif row['Cohort'] == 2:
        return 60, 1080
    else:
        return np.nan, np.nan
    
# Convert 'Cohort' to integer
df['Cohort'] = df['Cohort'].astype(int)

# Apply the function to create the 'Framerate' and 'y_pixels' columns
df[['Framerate', 'y_pixels']] = df.apply(set_framerate_and_ypixels, axis=1, result_type='expand')

In [12]:
def calculate_weighted_position(row):
    if row['TailBase_to_end'] >= 75:
        return row['Start_y_adjust']
    elif row['TailBase_to_end'] <= 0:
        return row['End_y_adjust']
    else:
        weight_start = (row['TailBase_to_end'] / 75)
        weight_end = 1 - weight_start
        return (weight_start * row['Start_y_adjust']) + (weight_end * row['End_y_adjust'])

# Apply the function to create the 'Position_y' column
df['Position_y'] = df.apply(calculate_weighted_position, axis=1)

In [13]:
#Get distance to end
df['Nose_to_end'] = 75 - (df['Nose_x']-df['Start_x_adjust'])/((df['End_x_adjust']-df['Start_x_adjust'])/75)
df['TailBase_to_end'] = 75 - (df['TailBase_x']-df['Start_x_adjust'])/((df['End_x_adjust']-df['Start_x_adjust'])/75)

#Get distance traveled
df['Nose_diff'] = df['Nose_to_end'].diff().abs()
df['TailBase_diff'] = df['TailBase_to_end'].diff().abs()
df['Distance_traveled'] = (df['Nose_diff'] + df['TailBase_diff']) / 2

#Determine speed
df['Speed_cm/s'] = df['Distance_traveled']*df['Framerate']

# Calculate the condition for 'Start_box_nose' and set to False when the condition is not met
threshold = df['Start_x_adjust'] + (10 * ((df['End_x_adjust'] - df['Start_x_adjust']) / 75))
df['Start_box_nose'] = (df['Nose_x'] - df['Start_x_adjust']) < threshold
df['Start_box_nose'] = df['Start_box_nose'].fillna(False)

# Calculate the condition for 'Start_box_tailbase' and set to False when the condition is not met
df['Start_box_tailbase'] = (df['TailBase_x'] - df['Start_x_adjust']) < threshold
df['Start_box_tailbase'] = df['Start_box_tailbase'].fillna(False)

#Pawslips
df['FrontLimb_slip'] = df['FrontLimb_y'] > df['Position_y']
df['FrontLimb_slip'] = df['FrontLimb_slip'].fillna(False)
df['HindLimb_slip'] = df['HindLimb_y'] > df['Position_y']
df['HindLimb_slip'] = df['HindLimb_slip'].fillna(False)
#df['Dragging'] = df['TailBase_y'] > ((df['Start_y_adjust'] + df['End_y_adjust'])/2)+((df['y_pixels'].apply(lambda x: 10 if x == 720 else (x / 720 * 10 if x == 1080 else None))))


In [14]:
df[['TailBase_to_end', 'Start_y_adjust','End_y_adjust', 'Position_y']].head(50)

1,TailBase_to_end,Start_y_adjust,End_y_adjust,Position_y
3,75.456465,235.666721,219.110961,235.666721
4,73.429442,235.666721,219.110961,235.320030
5,73.429533,235.666721,219.110961,235.320050
6,71.530275,235.666721,219.110961,234.900801
7,71.010160,235.666721,219.110961,234.785990
8,71.010273,235.666721,219.110961,234.786014
9,68.829003,235.666721,219.110961,234.304513
10,66.740851,235.666721,219.110961,233.843567
11,66.731339,235.666721,219.110961,233.841468
12,64.055454,235.666721,219.110961,233.250784


### Functions

In [4]:
def preprocess(df):

    """
    Takes the input file and returns a dataframe with the angles (both corrected and uncorrected)
    Parameters
    ----------

    df : pandas dataframe
        output .csv from the deeplabcut analysis.

    Returns
    -------
    df : pandas dataframe
        Same dataframe as input cleaner with generlized column names.
    """

    #seperate x, y and likelihood into 3 dfs for renaming and preprocessing
    df_x = df[df.columns[[1, 4, 7, 10, 13, 16]]]
    df_y = df[df.columns[[2, 5, 8, 11, 14, 17]]]
    df_like = df[df.columns[[3, 6, 9, 12, 15, 18]]]

    #drop the unneeded cols
    df_x = df_x.drop(0)
    df_y = df_y.drop(0)
    df_x = df_x.drop(2)
    df_y = df_y.drop(2)

    df_like = df_like.drop(0)
    df_like = df_like.drop(2)

    #Rename variables
    df_x.columns=df_x.iloc[0]
    df_x = df_x.drop(1)
    df_x.rename(columns = {'Nose':'Nose_x', 'Frontlimb':'FrontLimb_x', 'HindLimb':'HindLimb_x', 'TailBase':'TailBase_x', 'Start':'Start_x', 'End':'End_x'}, inplace = True)


    df_y.columns=df_y.iloc[0]
    df_y = df_y.drop(1)
    df_y.rename(columns = {'Nose':'Nose_y', 'Frontlimb':'FrontLimb_y', 'HindLimb':'HindLimb_y', 'TailBase':'TailBase_y', 'Start':'Start_y', 'End':'End_y'}, inplace = True)


    df_like.columns=df_like.iloc[0]
    df_like = df_like.drop(1)
    df_like.rename(columns = {'Nose':'Nose_likelihood', 'Frontlimb':'FrontLimb_likelihood', 'HindLimb':'HindLimb_likelihood', 'TailBase':'TailBase_likelihood', 'Start':'Start_likelihood', 'End':'End_likelihood'}, inplace = True)

    #merge the dataframes
    df_all = df_x.join(df_y)
    df_all = df_all.join(df_like)
    df_all = df_all.astype(float)

    return df_all

 

def files_from_folder(path):

    """
    Takes the input path and returns a dataframe with the concatanated data found in the folder

    Parameters
    ----------
    Path : input path for the folder where the files to be loaded are

    Returns
    ------
    df : pandas dataframe
        A concatanated dataframe with all the values
    """

    import glob
    import os.path
    import pandas as pd

    csv_files = glob.glob(os.path.join(path, "*.csv"))
    df = []

    # loop over the list of csv files
    for f in csv_files:

        # read the csv file
        file = pd.read_csv(f, header = None)
        file = preprocess(file)
        file['Name File'] = os.path.basename(f)
        file['Name File'] = file['Name File'].str.split('DLC_').str[0]
        df.append(file)
    df_con = pd.concat(df)

    return df_con